In [4]:
import sys
import urllib
import requests
import numpy as np
import pandas as pd
from io import StringIO
from pandas import json_normalize

from pathlib import Path

sys.path.append(str(Path.cwd().parent / 'scripts'))

from _helpers import to_date_period, to_datetime
from _elexon_helpers import robust_request
# from build_base import build_physical_notifications_period

date, period = '2024-03-23', 24

In [5]:
accepts_url = (
    'https://data.elexon.co.uk/bmrs/api/v1/balancing/acceptances/' +
    'all?settlementDate={}&settlementPeriod={}&format=csv'
    )

# prices_url = (
#     "https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/"
#     "indicative/volumes/all/{}/{}/{}?{}&format=json"
# )

bidsoffers_url = (
    "https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/"
    "indicative/volumes/all/{}/{}/{}?format=json"
)

trades_url = (
        "https://data.elexon.co.uk/bmrs/api/v1/balancing/bid-offer/" +
        "all?settlementDate={}&settlementPeriod={}&format=csv".format(date, period)
    )

In [6]:
response = robust_request(requests.get, accepts_url.format(date, period))
accepts = pd.read_csv(StringIO(response.text))
print(f"Retrieved {len(accepts)} balancing actions.")

Retrieved 247 balancing actions.


In [7]:
accepts.head()

,SettlementDate,SettlementPeriodFrom,SettlementPeriodTo,TimeFrom,TimeTo,LevelFrom,LevelTo,NationalGridBmUnit,BmUnit,AcceptanceNumber,AcceptanceTime,DeemedBoFlag,SoFlag,StorFlag,RrFlag
0,2024-03-23,24,25,2024-03-23T11:59:00Z,2024-03-23T12:14:00Z,400,400,CARR-1,T_CARR-1,102062,2024-03-23T11:52:00Z,False,False,False,False
1,2024-03-23,24,25,2024-03-23T11:59:00Z,2024-03-23T12:14:00Z,700,700,KEAD-2,T_KEAD-2,15844,2024-03-23T11:50:00Z,False,True,False,False
2,2024-03-23,24,25,2024-03-23T11:59:00Z,2024-03-23T12:10:00Z,25,25,GLOFW-1,E_GLOFW-1,45149,2024-03-23T11:56:00Z,False,False,False,False
3,2024-03-23,24,25,2024-03-23T11:59:00Z,2024-03-23T12:09:00Z,360,360,SCCL-2,T_SCCL-2,117669,2024-03-23T11:55:00Z,False,False,False,False
4,2024-03-23,24,25,2024-03-23T11:59:00Z,2024-03-23T12:01:00Z,-12,0,KEMB-1,T_KEMB-1,12666,2024-03-23T11:25:00Z,False,False,False,False


In [8]:
def fetch_acceptances(date_str, period_str):
    """
    Fetch Balancing Acceptances data from ELEXON API.
    """
    acceptances_url = (
        'https://data.elexon.co.uk/bmrs/api/v1/balancing/acceptances/' +
        'all?settlementDate={}&settlementPeriod={}&format=csv'
    ).format(date_str, period_str)
    
    response = requests.get(acceptances_url)
    response.raise_for_status()  # Raise an error for bad status codes
    
    acceptances_df = pd.read_csv(StringIO(response.text))
    return acceptances_df


acc = fetch_acceptances(date, period)

corrected = []

for bm in acc.NationalGridBmUnit.unique():

    ss = acc.loc[acc.NationalGridBmUnit == bm]
    ss = ss.loc[ss.AcceptanceTime == ss.AcceptanceTime.max()]

    corrected.append(ss)

acc = pd.concat(corrected)
# acc = acc.loc[acc.SoFlag]

# acc.sort_values(by='LevelFrom').head(20)
# acc.loc[acc.NationalGridBmUnit == 'SGRWO-1']

In [9]:
def fetch_trades(date_str, period_str, accepted_units):
    """
    Fetch Bid-Offer Prices data from ELEXON API for the accepted BM Units.
    """
    trades_url = (
        'https://data.elexon.co.uk/bmrs/api/v1/balancing/bid-offer/' +
        'all?settlementDate={}&settlementPeriod={}&format=csv'
    ).format(date_str, period_str)
    
    response = requests.get(trades_url)
    response.raise_for_status()
    
    trades_df = pd.read_csv(StringIO(response.text))
    # Filter for accepted BM Units
    trades_df = trades_df[trades_df['NationalGridBmUnit'].isin(accepted_units)]
    return trades_df

In [10]:
def fetch_volumes(date_str, period_str, accepted_units, data_type):
    """
    Fetch Bid or Offer Volumes data from ELEXON API.
    data_type should be 'bid' or 'offer'.
    """
    # Prepare BM Units for the API request
    unit_params = '&'.join(f"bmUnit={urllib.parse.quote_plus(unit)}" for unit in accepted_units)
    
    volumes_url = (
        "https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/"
        f"indicative/volumes/all/{data_type}/{date_str}/{period_str}?format=json&{unit_params}"
    )
    
    response = requests.get(volumes_url)
    response.raise_for_status()
    
    volumes_json = response.json()
    if 'data' in volumes_json:
        volumes_df = pd.json_normalize(volumes_json['data'])
    else:
        volumes_df = pd.DataFrame()
    return volumes_df

In [11]:
def merge_data(acceptances_df, volumes_df, trades_df):
    """
    Merge acceptances, volumes, and trades data into a single DataFrame.
    """
    # Merge acceptances with volumes
    merged_df = acceptances_df.merge(
        volumes_df,
        left_on=['SettlementDate', 'SettlementPeriodFrom', 'BmUnit'],
        right_on=['settlementDate', 'settlementPeriod', 'bmUnit'],
        how='left'
    )
    # print(merged_df.columns)
    # print(trades_df.columns)
    # print(merged_df.shape)
    # print(trades_df.shape)
    # print(trades_df.head())
    
    # Merge with trades to get prices
    final_df = merged_df.merge(
        trades_df,
        left_on=['settlementDate', 'settlementPeriod', 'NationalGridBmUnit', 'bmUnit'],
        right_on=['SettlementDate', 'SettlementPeriod', 'NationalGridBmUnit', 'BmUnit'],
        how='left',
        suffixes=('_acceptance', '_trade')
    )
    return final_df

In [12]:
def clean_and_format_data(final_df):
    """
    Select relevant columns, rename them, and clean the DataFrame.
    """
    # Select and rename columns for clarity
    final_df = final_df[[
        'SettlementDate_acceptance', 'SettlementPeriodFrom', 'TimeFrom_acceptance', 'TimeTo_acceptance',
        'NationalGridBmUnit', 'AcceptanceNumber', 'AcceptanceTime',
        'totalVolumeAccepted', 'Bid', 'Offer', 'PairId'
    ]]

    final_df = final_df.rename(columns={
        'SettlementPeriodFrom': 'SettlementPeriod',
        'TimeFrom_acceptance': 'TimeFrom',
        'SettlementDate_acceptance': 'SettlementDate',
        'TimeTo_acceptance': 'TimeTo',
        'totalVolumeAccepted': 'VolumeAccepted'
    })

    # Remove duplicates if any
    final_df.drop_duplicates(inplace=True)

    # Handle missing values (optional)
    final_df.fillna({'Bid': 0, 'Offer': 0, 'VolumeAccepted': 0}, inplace=True)

    return final_df

In [13]:
def build_bm_data_period(date, period):

    acceptances_df = fetch_acceptances(date, period)
    acceptances_df = acceptances_df.loc[acceptances_df['SoFlag']]

    accepted_units = acceptances_df['NationalGridBmUnit'].unique().tolist()

    trades_df = fetch_trades(date, period, accepted_units)

    bids_df = fetch_volumes(date, period, accepted_units, data_type='bid')    
    offers_df = fetch_volumes(date, period, accepted_units, data_type='offer')

    volumes_df = pd.concat([bids_df, offers_df], ignore_index=True)

    return acceptances_df, trades_df, volumes_df


acc, tra, vol = build_bm_data_period(date, period)

In [16]:
bm = 'CARR-1'

In [19]:
t = tra.loc[tra.NationalGridBmUnit == bm]
v = vol.loc[vol.nationalGridBmUnit == bm]

In [36]:
true_cap = 176 # MW
true_price = 84 # £/MWh
true_revenue = 9_015.75 # £

(c_vol := (true_cap * 0.5 + (400 - true_cap) * 8 / 60 / 2 + (400 - true_cap) / 60)) * true_price

8960.0

In [37]:
c_vol

106.66666666666667

In [41]:
acc.columns

Index(['SettlementDate', 'SettlementPeriodFrom', 'SettlementPeriodTo',
       'TimeFrom', 'TimeTo', 'LevelFrom', 'LevelTo', 'NationalGridBmUnit',
       'BmUnit', 'AcceptanceNumber', 'AcceptanceTime', 'DeemedBoFlag',
       'SoFlag', 'StorFlag', 'RrFlag'],
      dtype='object')

In [84]:
offer_cols = vol.columns[vol.columns.str.contains('positive')].tolist()
bid_cols = vol.columns[vol.columns.str.contains('negative')].tolist()

detected_offers = list()
detected_bids = list()

for bm in acc.NationalGridBmUnit.unique():

    ss = vol.loc[vol.nationalGridBmUnit == bm]
    t = tra.loc[tra.NationalGridBmUnit == bm]

    offer_row = ss[['dataType'] + offer_cols]
    offer_row = (
        offer_row.loc[offer_row['dataType'] == 'Tagged', offer_cols]
        .fillna(value=0)
        .abs()
        .max()
    )

    bid_row = ss[['dataType'] + bid_cols]
    bid_row = (
        bid_row.loc[bid_row['dataType'] == 'Tagged', bid_cols]
        .fillna(value=0)
        .abs()
        .max()
    )

    offer_trade = t.loc[t.PairId > 0, 'Offer']
    bid_trade = t.loc[t.PairId < 0, 'Bid'].iloc[::-1]

    for trade_price, trade_vol in zip(offer_trade, offer_row.loc[offer_row != 0].values):
        detected_offers.append(pd.Series({
            'vol': trade_vol,
            'price': trade_price
        }, name=bm))
    
    for trade_price, trade_vol in zip(bid_trade, bid_row.loc[bid_row != 0].values):
        detected_bids.append(pd.Series({
            'vol': trade_vol,
            'price': trade_price
        }, name=bm))


NationalGridBmUnit
KEAD-2     9
ASHWW-1    7
DIDCB5     6
SGRWO-1    5
PEMB-51    5
Name: count, dtype: int64
['pairVolumes.positive1', 'pairVolumes.positive2', 'pairVolumes.positive3', 'pairVolumes.positive4', 'pairVolumes.positive5', 'pairVolumes.positive6']
['pairVolumes.negative1', 'pairVolumes.negative2', 'pairVolumes.negative3', 'pairVolumes.negative4', 'pairVolumes.negative5', 'pairVolumes.negative6']
KEAD-2
ASHWW-1
CUPAB-1
SGRWO-4
BHLAW-1
SGRWO-5
SGRWO-3
SGRWO-1
BLARW-1
AKGLW-2
SGRWO-6
CRMLW-1
DIDCB5
CRDEW-1
CRDEW-2
PEMB-51
CARR-1
BABAW-1
GLWSW-1
TWSHW-1
WDRGW-1
BTUIW-3
ASLVW-1
AKGLW-3
SGRWO-2
GLOFW-1
HALSW-1
GORDW-2
LCSMH-1
BRYBW-1
CUMHW-1
CAS-GAR01


In [85]:
total_offers = pd.concat(detected_offers, axis=1).T
total_offers['revenue'] = total_offers.vol * total_offers.price

total_offers

,vol,price,revenue
KEAD-2,224.206250,81.0,18160.70625
DIDCB5,190.000000,99.0,18810.00000
PEMB-51,109.500000,137.0,15001.50000
CARR-1,107.488083,84.0,9028.99900


In [86]:
total_bids = pd.concat(detected_bids, axis=1).T
total_bids['revenue'] = total_bids.vol * total_bids.price

total_bids

,vol,price,revenue
ASHWW-1,17.000000,-75.00,-1275.000000
CUPAB-1,9.182917,-22.60,-207.533917
CUPAB-1,0.150417,-22.60,-3.399417
SGRWO-4,67.865507,-27.02,-1833.725998
BHLAW-1,41.000000,-86.93,-3564.130000
SGRWO-5,38.919317,-27.02,-1051.599944
SGRWO-3,77.838634,-27.02,-2103.199889
SGRWO-1,140.713418,-27.02,-3802.076567
BLARW-1,17.500000,-73.00,-1277.500000
AKGLW-2,12.075000,-87.00,-1050.525000


In [79]:
date, period

('2024-03-23', 24)

In [240]:
final_df = merge_data(acc, vol, tra)

In [265]:
def filter_dummy_prices(df):
    assert 'Price' in df.columns, "Price column not found in the DataFrame."

    return df.loc[
        (df.Price != 0) &
        (df.Price.abs() < 999)
    ]

In [266]:
ff = clean_and_format_data(final_df)

ff['Action'] = np.nan
ff.loc[ff.PairId > 0, 'Action'] = 'Offer'
ff.loc[ff.PairId < 0, 'Action'] = 'Bid'

ff['Price'] = ff[['Bid', 'Offer']].max(axis=1)

corrected = list()

for bm in accepted_units:
    ss = ff.loc[ff.NationalGridBmUnit == bm]
    ff.drop(ss.index, inplace=True)
    ss = ss.loc[ss.AcceptanceTime == ss.AcceptanceTime.max()]
    ss = filter_dummy_prices(ss)

    corrected.append(ss)

ff = pd.concat(corrected)

In [274]:
accepted_units = ff.NationalGridBmUnit.unique()

In [275]:
import pandas as pd

# for bm in ['WBURB-1', 'COSO-1', 'DIDCB6', 'WBURB-2', 'FOYE-2', 'HUMR-1'][::-1]:
# for bm in ['HUMR-1'][::-1]:
for bm in accepted_units[:5]:

    ss = ff.loc[ff.NationalGridBmUnit.isin([bm])]
    ss = ss.loc[ss.AcceptanceTime == ss.AcceptanceTime.max()]
    # Assume 'df' is your provided DataFrame
    # df = your_dataframe  # Replace with your actual DataFrame variable

    # Step 1: Filter rows where VolumeAccepted is not zero
    df_nonzero = ss[ss['VolumeAccepted'] != 0].copy()

    # Step 2: Correct the 'Price' column based on 'Action'
    df_nonzero['Price'] = df_nonzero.apply(
        lambda row: row['Bid'] if row['Action'] == 'Bid' else row['Offer'], axis=1
    )

    # Step 3: Group by 'NationalGridBmUnit', 'Price', and 'Action' and sum 'VolumeAccepted'
    summary = df_nonzero.groupby(
        ['NationalGridBmUnit', 'Price', 'Action']
    )['VolumeAccepted'].sum().reset_index()

    # Step 4: Filter for bids (since your result is for bids)
    summary_bids = summary[summary['Action'] == 'Bid']
    
    summary_offers = summary[summary['Action'] == 'Offer']
    # print(summary_offers)

    final_result_bids = summary_bids[['NationalGridBmUnit', 'VolumeAccepted', 'Price', 'Action']]
    final_result_bids.rename(columns={'VolumeAccepted': 'volume'}, inplace=True)
    final_result_bids['revenue'] = final_result_bids['volume'].abs().mul(final_result_bids['Price'])
    final_result_bids = final_result_bids.loc[final_result_bids.revenue == final_result_bids.revenue.max()]

    # print(summary_offers)
    final_result_offers = summary_offers[['NationalGridBmUnit', 'VolumeAccepted', 'Price', 'Action']]
    final_result_offers.rename(columns={'VolumeAccepted': 'volume'}, inplace=True)
    final_result_offers['revenue'] = final_result_offers['volume'].abs().mul(final_result_offers['Price'])
    final_result_offers = final_result_offers.loc[final_result_offers.revenue == final_result_offers.revenue.max()]


    # print(pd.concat([final_result_bids, final_result_offers]))
    print(final_result_bids)
    print('============')
    print(final_result_offers)
    print('-------------------')


  NationalGridBmUnit      volume  Price Action      revenue
1             CARR-1  214.466679   41.0    Bid  8793.133853
  NationalGridBmUnit      volume  Price Action      revenue
3             CARR-1  214.466679  135.0  Offer  28953.00171
-------------------
  NationalGridBmUnit    volume  Price Action    revenue
1             KEAD-2  223.7875   50.0    Bid  11189.375
  NationalGridBmUnit    volume  Price Action     revenue
2             KEAD-2  223.7875   81.0  Offer  18126.7875
-------------------
  NationalGridBmUnit     volume  Price Action      revenue
0            GLOFW-1 -15.333333 -84.64    Bid -1297.813333
  NationalGridBmUnit     volume  Price Action      revenue
1            GLOFW-1 -15.333333  400.0  Offer  6133.333333
-------------------
  NationalGridBmUnit  volume  Price Action  revenue
1             SCCL-2     2.0   44.0    Bid     88.0
  NationalGridBmUnit  volume  Price Action  revenue
2             SCCL-2     2.0   83.0  Offer    166.0
-------------------
  National

In [286]:
vol.sort_values(by='totalVolumeAccepted').tail(30)

,createdDateTime,settlementDate,settlementPeriod,startTime,bmUnit,bmUnitType,leadPartyName,nationalGridBmUnit,dataType,totalVolumeAccepted,...,pairVolumes.negative2,pairVolumes.positive2,pairVolumes.negative3,pairVolumes.positive3,pairVolumes.negative4,pairVolumes.positive4,pairVolumes.negative5,pairVolumes.positive5,pairVolumes.negative6,pairVolumes.positive6
202,2024-03-24T12:14:30Z,2024-03-23,24,2024-03-23T11:30:00Z,T_SGRWO-6,T,Seagreen Wind Energy Limited,SGRWO-6,Re-priced,0.000000,...,NaN,NaN,None,NaN,None,None,None,None,None,None
217,2024-03-24T12:14:30Z,2024-03-23,24,2024-03-23T11:30:00Z,T_WDRGW-1,T,Windy Rig Wind Farm Limited,WDRGW-1,Original-Priced,0.000000,...,NaN,NaN,None,NaN,None,None,None,None,None,None
216,2024-03-24T12:14:30Z,2024-03-23,24,2024-03-23T11:30:00Z,T_WDRGW-1,T,Windy Rig Wind Farm Limited,WDRGW-1,Original,0.000000,...,NaN,NaN,None,NaN,None,None,None,None,None,None
214,2024-03-24T12:14:30Z,2024-03-23,24,2024-03-23T11:30:00Z,T_WBURB-43,T,West Burton B Limited,WBURB-43,Re-priced,0.000000,...,NaN,NaN,None,NaN,None,None,None,None,None,None
179,2024-03-24T12:14:30Z,2024-03-23,24,2024-03-23T11:30:00Z,T_SCCL-2,T,SCCL,SCCL-2,Tagged,0.000000,...,0.0,NaN,None,NaN,None,None,None,None,None,None
213,2024-03-24T12:14:30Z,2024-03-23,24,2024-03-23T11:30:00Z,T_WBURB-43,T,West Burton B Limited,WBURB-43,Original-Priced,0.000000,...,0.0,0.0,None,NaN,None,None,None,None,None,None
5,2024-03-24T12:14:30Z,2024-03-23,24,2024-03-23T11:30:00Z,2__GLOND001,S,EDF Energy Customers Ltd,AG-GEDF01,Original-Priced,0.000000,...,0.0,0.0,None,NaN,None,None,None,None,None,None
6,2024-03-24T12:14:30Z,2024-03-23,24,2024-03-23T11:30:00Z,2__GLOND001,S,EDF Energy Customers Ltd,AG-GEDF01,Re-priced,0.000000,...,NaN,NaN,None,NaN,None,None,None,None,None,None
13,2024-03-24T12:14:30Z,2024-03-23,24,2024-03-23T11:30:00Z,V__GHABI001,V,Habitat Energy Limited,AG-HEL00G,Original-Priced,0.000000,...,0.0,NaN,None,NaN,None,None,None,None,None,None
210,2024-03-24T12:14:30Z,2024-03-23,24,2024-03-23T11:30:00Z,T_WBURB-41,T,West Burton B Limited,WBURB-41,Re-priced,0.000000,...,NaN,NaN,None,NaN,None,None,None,None,None,None


In [167]:
ff['abs_vol'] = ff['VolumeAccepted'].abs()
this = ff.groupby(['SettlementDate', 'SettlementPeriod', 'NationalGridBmUnit', 'Action', 'Price'], as_index=False)['abs_vol'].max()
this['product'] = this['Price'] * this['abs_vol']

this

,SettlementDate,SettlementPeriod,NationalGridBmUnit,Action,Price,abs_vol,product
0,2021-01-10,24,AG-DSTK01,Bid,85.5,4.3500,371.92500
1,2021-01-10,24,AG-DSTK01,Offer,92.5,4.3500,402.37500
2,2021-01-10,24,AG-ESTK01,Bid,84.7,4.8333,409.38051
3,2021-01-10,24,AG-ESTK01,Offer,91.7,4.8333,443.21361
4,2021-01-10,24,AG-EUKP01,Bid,95.0,1.0000,95.00000
5,2021-01-10,24,AG-EUKP01,Offer,95.0,1.0000,95.00000
6,2021-01-10,24,AG-GSTK01,Bid,82.4,4.8333,398.26392
7,2021-01-10,24,AG-GSTK01,Offer,92.4,4.8333,446.59692
8,2021-01-10,24,AG-GSTK02,Bid,82.4,4.8333,398.26392
9,2021-01-10,24,AG-GSTK02,Offer,92.4,4.8333,446.59692


In [168]:
df = pd.DataFrame({
    'nationalGridBmUnit': ['T_WBURB-1', 'T_DIDCB6', 'T_COSO-1', 'T_WBURB-2', 'T_FOYE-2', 'T_HUMR-1'],
    'volume': [-20., -55.98, -26.53, -15.05, 40.20, 57.97],
    'price': [46.2, 46.1, 46.22, 46.2, 115., 74.4],
    'Action': ['Bid', 'Bid', 'Bid', 'Bid', 'Offer', 'Offer'],
})

df

,nationalGridBmUnit,volume,price,Action
0,T_WBURB-1,-20.00,46.20,Bid
1,T_DIDCB6,-55.98,46.10,Bid
2,T_COSO-1,-26.53,46.22,Bid
3,T_WBURB-2,-15.05,46.20,Bid
4,T_FOYE-2,40.20,115.00,Offer
5,T_HUMR-1,57.97,74.40,Offer


['T_WBURB-1', 'T_DIDCB6', 'T_COSO-1', 'T_WBURB-2', 'T_FOYE-2', 'T_HUMR-1']

In [148]:
for bm in ff.NationalGridBmUnit.unique():
    # print(bm)
    # print(ff.loc[ff.NationalGridBmUnit == bm, 'VolumeAccepted'].sum())

    ss = ff.loc[ff.NationalGridBmUnit == 'ROCK-1']

    off = ss.loc[ss.PairId > 0]

    # print('offers')
    # print(ss['VolumeAccepted'].mul(ss['Offer']).sum() / ss['VolumeAccepted'].sum())
    # print(ss['VolumeAccepted'].sum())

    # print(ss)

    off = pd.Series({
        'VolumeAccepted': off['VolumeAccepted'].sum(),
        'Offer': off['VolumeAccepted'].mul(off['Offer']).sum() / off['VolumeAccepted'].sum()
    })

    bid = ss.loc[ss.PairId < 0]
    bid = pd.Series({
        'VolumeAccepted': bid['VolumeAccepted'].sum(),
        'Bid': bid['VolumeAccepted'].mul(bid['Bid']).sum() / bid['VolumeAccepted'].sum()
    })

    # print(off, bid)
    break

# ff.loc[ff.NationalGridBmUnit == 'ROCK-1']

ss.loc[ss.PairId > 0, 'Action'] = "Offer"
ss.loc[ss.PairId < 0, 'Action'] = "Bid"

print(ss.head())
ss.groupby([
    'SettlementDate', 'SettlementPeriod', 'NationalGridBmUnit',
    'Action', 'Price'
], as_index=False)['Volume'].sum()

    SettlementDate  SettlementPeriod              TimeFrom  \
568     2021-01-10                24  2021-01-10T11:47:00Z   
569     2021-01-10                24  2021-01-10T11:47:00Z   
570     2021-01-10                24  2021-01-10T11:47:00Z   
580     2021-01-10                24  2021-01-10T11:47:00Z   
581     2021-01-10                24  2021-01-10T11:47:00Z   

                   TimeTo NationalGridBmUnit  AcceptanceNumber  \
568  2021-01-10T12:00:00Z             ROCK-1            174414   
569  2021-01-10T12:00:00Z             ROCK-1            174414   
570  2021-01-10T12:00:00Z             ROCK-1            174414   
580  2021-01-10T12:00:00Z             ROCK-1            174414   
581  2021-01-10T12:00:00Z             ROCK-1            174414   

           AcceptanceTime  VolumeAccepted   Bid  Offer  PairId Action  
568  2021-01-10T10:33:00Z            0.00   0.0    0.5    -2.0    Bid  
569  2021-01-10T10:33:00Z            0.00   1.0   25.0    -1.0    Bid  
570  2021-01-1

C:\Users\lukas\AppData\Local\Temp\ipykernel_37700\2166013869.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss.loc[ss.PairId > 0, 'Action'] = "Offer"


KeyError: 'Price'

In [98]:
f = final_df.loc[~final_df['Bid'].isna()]

In [100]:
print(f.columns)
f.head()

Index(['SettlementDate_acceptance', 'SettlementPeriodFrom',
       'SettlementPeriodTo', 'TimeFrom_acceptance', 'TimeTo_acceptance',
       'LevelFrom_acceptance', 'LevelTo_acceptance', 'NationalGridBmUnit',
       'BmUnit_acceptance', 'AcceptanceNumber', 'AcceptanceTime',
       'DeemedBoFlag', 'SoFlag', 'StorFlag', 'RrFlag', 'createdDateTime',
       'settlementDate', 'settlementPeriod', 'startTime', 'bmUnit',
       'bmUnitType', 'leadPartyName', 'nationalGridBmUnit', 'dataType',
       'totalVolumeAccepted', 'pairVolumes.negative1', 'pairVolumes.positive1',
       'pairVolumes.negative2', 'pairVolumes.positive2',
       'pairVolumes.negative3', 'pairVolumes.positive3',
       'pairVolumes.negative4', 'pairVolumes.positive4',
       'pairVolumes.negative5', 'pairVolumes.positive5',
       'pairVolumes.negative6', 'pairVolumes.positive6',
       'SettlementDate_trade', 'SettlementPeriod', 'BmUnit_trade',
       'TimeFrom_trade', 'TimeTo_trade', 'LevelFrom_trade', 'LevelTo_trade',
   

,SettlementDate_acceptance,SettlementPeriodFrom,SettlementPeriodTo,TimeFrom_acceptance,TimeTo_acceptance,LevelFrom_acceptance,LevelTo_acceptance,NationalGridBmUnit,BmUnit_acceptance,AcceptanceNumber,...,SettlementDate_trade,SettlementPeriod,BmUnit_trade,TimeFrom_trade,TimeTo_trade,LevelFrom_trade,LevelTo_trade,Bid,Offer,PairId
0,2021-01-10,24,25,2021-01-10T11:59:00Z,2021-01-10T12:00:00Z,0,90,FFES-4,T_FFES-4,153149,...,2021-01-10,24.0,T_FFES-4,2021-01-10T11:30:00Z,2021-01-10T12:00:00Z,-165.0,-165.0,0.0,117.5,-1.0
1,2021-01-10,24,25,2021-01-10T11:59:00Z,2021-01-10T12:00:00Z,0,90,FFES-4,T_FFES-4,153149,...,2021-01-10,24.0,T_FFES-4,2021-01-10T11:30:00Z,2021-01-10T12:00:00Z,165.0,165.0,0.0,117.5,1.0
2,2021-01-10,24,25,2021-01-10T11:59:00Z,2021-01-10T12:00:00Z,0,90,FFES-4,T_FFES-4,153149,...,2021-01-10,24.0,T_FFES-4,2021-01-10T11:30:00Z,2021-01-10T12:00:00Z,-165.0,-165.0,0.0,117.5,-1.0
3,2021-01-10,24,25,2021-01-10T11:59:00Z,2021-01-10T12:00:00Z,0,90,FFES-4,T_FFES-4,153149,...,2021-01-10,24.0,T_FFES-4,2021-01-10T11:30:00Z,2021-01-10T12:00:00Z,165.0,165.0,0.0,117.5,1.0
4,2021-01-10,24,25,2021-01-10T11:59:00Z,2021-01-10T12:00:00Z,0,90,FFES-4,T_FFES-4,153149,...,2021-01-10,24.0,T_FFES-4,2021-01-10T11:30:00Z,2021-01-10T12:00:00Z,-165.0,-165.0,0.0,117.5,-1.0


In [230]:
# def get_bm(bm, start, end):
def get_bm(bm, date, period):

    dt = pd.to_datetime(date).strftime('%Y-%m-%d')
    end_period = period + 1

    # bm_url = f'https://data.elexon.co.uk/bmrs/api/v1/balancing/acceptances?bmUnit={bm}&from={start}T00%3A00Z&to={end}T00%3A00Z&settlementPeriodFrom=1&settlementPeriodTo=48&format=json'
    bm_url = (
        'https://data.elexon.co.uk/bmrs/api/v1/balancing/acceptances'
        # f'?bmUnit={bm}&from={dt}T00%3A00Z&to={dt}T00%3A00Z&settlementPeriodFrom={period}'
        # f'&settlementPeriodTo={end_period}&format=json'
        f'?bmUnit={bm}&from={dt}T00%3A00Z&to={dt}T00%3A00Z&settlementPeriodFrom=1'
        f'&settlementPeriodTo=48&format=json'
    )

    bm_request = requests.get(bm_url)
    bm_response = bm_request.json()

    bm_raw = pd.concat([pd.Series(r) for r in bm_response['data']], axis=1).T
    bm_raw['timeFrom'] = pd.to_datetime(bm_raw.timeFrom)
    bm_raw['timeTo'] = pd.to_datetime(bm_raw.timeTo)
    bm_raw = bm_raw.sort_values('timeFrom').set_index('timeFrom')
    bm_raw.index = pd.to_datetime(bm_raw.index)

    print(bm_raw.columns)
    bm_raw = (
        bm_raw
        .reset_index()
        .sort_values(['acceptanceTime', 'timeFrom'])
        [[
            'settlementDate', 'settlementPeriodFrom', 'settlementPeriodTo',
            'timeFrom', 'timeTo', 'levelFrom', 'levelTo',
            'acceptanceTime', 'deemedBoFlag', 'soFlag', 'storFlag', 'rrFlag'
            ]]
    )

    return bm_raw

dt = to_datetime(date, period)
bm = 'HUMR-1'
print(dt)
bm_df = get_bm(bm, date, period)



2021-01-10 11:30:00+00:00
Index(['settlementDate', 'settlementPeriodFrom', 'settlementPeriodTo',
       'timeTo', 'levelFrom', 'levelTo', 'nationalGridBmUnit', 'bmUnit',
       'acceptanceNumber', 'acceptanceTime', 'deemedBoFlag', 'soFlag',
       'storFlag', 'rrFlag'],
      dtype='object')


In [226]:
tra = pd.concat([
    fetch_trades(date, i, ['HUMR-1'])
    for i in range(1, 49)
])

In [228]:
tra.loc[tra.TimeTo <= '2021-01-10T00:30:00Z']

,SettlementDate,SettlementPeriod,NationalGridBmUnit,BmUnit,TimeFrom,TimeTo,LevelFrom,LevelTo,Bid,Offer,PairId
483,2021-01-10,1,HUMR-1,T_HUMR-1,2021-01-10T00:00:00Z,2021-01-10T00:30:00Z,-40,-40,-9999.0,250.0,-4
484,2021-01-10,1,HUMR-1,T_HUMR-1,2021-01-10T00:00:00Z,2021-01-10T00:30:00Z,-345,-345,-50.0,250.0,-3
485,2021-01-10,1,HUMR-1,T_HUMR-1,2021-01-10T00:00:00Z,2021-01-10T00:30:00Z,-336,-336,-50.0,250.0,-2
486,2021-01-10,1,HUMR-1,T_HUMR-1,2021-01-10T00:00:00Z,2021-01-10T00:30:00Z,-1,-1,-50.0,250.0,-1
487,2021-01-10,1,HUMR-1,T_HUMR-1,2021-01-10T00:00:00Z,2021-01-10T00:30:00Z,1,1,-50.0,250.0,1
488,2021-01-10,1,HUMR-1,T_HUMR-1,2021-01-10T00:00:00Z,2021-01-10T00:30:00Z,3,3,-50.0,250.0,2
489,2021-01-10,1,HUMR-1,T_HUMR-1,2021-01-10T00:00:00Z,2021-01-10T00:30:00Z,51,51,-50.0,250.0,3
490,2021-01-10,1,HUMR-1,T_HUMR-1,2021-01-10T00:00:00Z,2021-01-10T00:30:00Z,547,547,-50.0,250.0,4


In [232]:
bm_df.loc[bm_df.timeTo <= '2021-01-10T00:30:00Z']

,settlementDate,settlementPeriodFrom,settlementPeriodTo,timeFrom,timeTo,levelFrom,levelTo,acceptanceTime,deemedBoFlag,soFlag,storFlag,rrFlag


In [109]:
from functools import wraps

_cache = {}
_calls_remaining = {}


def get_a(date, period):
    print('getting a')
    return f'a_{date}_{period}'

def get_x(date, period):
    print('getting x')
    return f'x_{date}_{period}'


def cache_a_x(func):
    @wraps(func)
    def wrapper(date, period):
        key = (date, period)
        if key not in _cache:
            print('building a, x')

            _cache[key] = {}
            _cache[key]['a'] = get_a(date, period)
            _cache[key]['x'] = get_x(date, period)

            _calls_remaining[key] = {'get_b': True, 'get_c': True}

        result = func(_cache[key]['a'], _cache[key]['x'], date, period)
        _cleanup(func.__name__, key)
        return result
    return wrapper

def _cleanup(func_name, key):
    _calls_remaining[key][func_name] = False
    if not any(_calls_remaining[key].values()):
        print("Deleting a and x...")
        del _cache[key]
        del _calls_remaining[key]

@cache_a_x
def get_b(a, x, date, period):
    print('getting b')
    return a + '_' + x + '_b'

@cache_a_x
def get_c(a, x, date, period):
    print('getting c')
    return a + '_' + x + '_c'

print(get_b('2020', 1))
print('-------------')
print(_cache)
print(_calls_remaining)
print('-------------')
print(get_c('2020', 1))

print('-------------')
print(_cache)
print(_calls_remaining)
print('-------------')
b = get_b('2020', 2)
print('-------------')
print(_cache)
print(_calls_remaining)
print('-------------')
c = get_c('2020', 2)
print('-------------')
print(_cache)
print(_calls_remaining)

building a, x
getting a
getting x
getting b
a_2020_1_x_2020_1_b
-------------
{('2020', 1): {'a': 'a_2020_1', 'x': 'x_2020_1'}}
{('2020', 1): {'get_b': False, 'get_c': True}}
-------------
getting c
Deleting a and x...
a_2020_1_x_2020_1_c
-------------
{}
{}
-------------
building a, x
getting a
getting x
getting b
-------------
{('2020', 2): {'a': 'a_2020_2', 'x': 'x_2020_2'}}
{('2020', 2): {'get_b': False, 'get_c': True}}
-------------
getting c
Deleting a and x...
-------------
{}
{}


In [94]:
from functools import wraps

# Shared cache and call tracking
_cache = {}
_calls_remaining = {'b': True, 'c': True}

def cache_a(func):
    @wraps(func)
    def wrapper():
        if 'a' not in _cache:
            # Build 'a' here
            print("Building 'a'...")
            _cache['a'] = {'data': 'This is a'}
        result = func(_cache['a'])
        _cleanup(func.__name__)
        return result
    return wrapper

def _cleanup(func_name):
    _calls_remaining[func_name] = False
    if not any(_calls_remaining.values()):
        print("Deleting 'a'...")
        del _cache['a']

@cache_a
def get_b(a):
    # Compute 'b' using 'a'
    print("Computing 'b' using 'a'...")
    return f"Result b with {a['data']}"

@cache_a
def get_c(a):
    # Compute 'c' using 'a'
    print("Computing 'c' using 'a'...")
    return f"Result c with {a['data']}"

# Usage
b = get_b()
print(b)
c = get_c()
print(c)




Building 'a'...
Computing 'b' using 'a'...
Result b with This is a
Computing 'c' using 'a'...
Result c with This is a
